In [1]:
import pandas as pd
pd.set_option('display.max_row', 500)
pd.set_option('display.max_columns', 100)
import numpy as np
import matplotlib.pyplot as plt  # 그래프 그리는 라이브러리
from sklearn.model_selection import train_test_split
from sklearn import metrics  # 평가를 위한 라이브러리
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC, LinearSVR, SVR

import seaborn as sns  # 시각화 라이브러리

### 날씨 데이터 가져와서 멀티 인덱스 설정

In [2]:
weather = pd.read_csv("baseball_data/2012~2018날씨데이터/2012_data.csv", encoding = "utf-8")
weather = weather.set_index(["day_text","location"])
weather

,,지점,기온(°C),강수량(mm),풍속(m/s),풍향(16방위),습도(%),현지기압(hPa),지면온도(°C)
day_text,location,,,,,,,,
2012-04-01 1:00,백령도,102,2.1,NaN,4.4,290.0,53.0,1004.3,0.9
2012-04-01 2:00,백령도,102,1.9,NaN,4.1,290.0,56.0,1004.3,0.7
2012-04-01 3:00,백령도,102,1.9,NaN,3.9,290.0,57.0,1004.2,0.4
2012-04-01 4:00,백령도,102,1.7,NaN,4.5,290.0,63.0,1004.3,0.2
2012-04-01 5:00,백령도,102,1.7,NaN,5.1,290.0,59.0,1004.7,0.1
...,...,...,...,...,...,...,...,...,...
2013-03-31 20:00,강화,201,4.2,NaN,3.8,250.0,44.0,1017.4,3.3
2013-03-31 21:00,강화,201,4.1,NaN,2.6,250.0,45.0,1017.9,2.5
2013-03-31 22:00,강화,201,3.5,NaN,1.6,250.0,40.0,1019.0,1.2


In [3]:
baseball = pd.read_csv("baseball_data/2012~2018야구데이터/2012.csv", encoding = "utf-8")
baseball = baseball.set_index(["day_text","location"])
baseball

day times_text team_text  H_text  HR_text  \
day_text         location                                                
2012-04-07 14:00 대전         4.7토      13:55    한화 이글스      11        0   
                 대전         4.7토      13:55   롯데 자이언츠      10        1   
                 광주         4.7토      13:55  KIA 타이거즈       6        0   
                 광주         4.7토      13:55   SK 와이번스      10        0   
                 서울         4.7토      14:00    LG 트윈스       9        1   
...                          ...        ...       ...     ...      ...   
2012-09-30 17:00 부산        9.30일      17:00  KIA 타이거즈       2        1   
                 서울        9.30일      17:00    두산 베어스      14        0   
                 서울        9.30일      17:00    LG 트윈스       9        0   
                 대구        9.30일      17:00   삼성 라이온즈      13        1   
                 대구        9.30일      17:00   키움 히어로즈       7        0   

                           S_text  ST_text  DU_text  ER_text result_text  
day_text         location                                                 
2012-04-07 14:00 대전             2        6        2        2           패  
                 대전             2        6        1        1           승  
                 광주             2        8        0        3           패  
                 광주             1        2        1        0           승  
                 서울             1        3        2        0           승  
...                           ...      ...      ...      ...         ...  
2012-09-30 17:00 부산             1        8        0        0           승  
                 서울             0        9        2        1           승  
                 서울             2        6        0        1           패  
                 대구             1        4        0        0           승  
                 대구             3        8        1        0           패  

[1026 rows x 10 columns]

In [4]:
pd.concat([baseball,weather], ignore_index=False, join="inner")

Empty DataFrame
Columns: []
Index: [(2012-04-07 14:00, 대전), (2012-04-07 14:00, 대전), (2012-04-07 14:00, 광주), (2012-04-07 14:00, 광주), (2012-04-07 14:00, 서울), (2012-04-07 14:00, 서울), (2012-04-07 14:00, 서울), (2012-04-07 14:00, 서울), (2012-04-08 14:00, 대전), (2012-04-08 14:00, 대전), (2012-04-08 14:00, 광주), (2012-04-08 14:00, 광주), (2012-04-08 14:00, 서울), (2012-04-08 14:00, 서울), (2012-04-08 14:00, 서울), (2012-04-08 14:00, 서울), (2012-04-11 19:00, 부산), (2012-04-11 19:00, 부산), (2012-04-11 19:00, 서울), (2012-04-11 19:00, 서울), (2012-04-11 19:00, 인천), (2012-04-11 19:00, 인천), (2012-04-11 19:00, 대구), (2012-04-11 19:00, 대구), (2012-04-12 19:00, 부산), (2012-04-12 19:00, 부산), (2012-04-12 19:00, 서울), (2012-04-12 19:00, 서울), (2012-04-12 19:00, 인천), (2012-04-12 19:00, 인천), (2012-04-12 19:00, 대구), (2012-04-12 19:00, 대구), (2012-04-13 19:00, 대전), (2012-04-13 19:00, 대전), (2012-04-13 19:00, 광주), (2012-04-13 19:00, 광주), (2012-04-13 19:00, 서울), (2012-04-13 19:00, 서울), (2012-04-13 19:00, 서울), (2012-04-13 19:00, 서울), (2012-04-14 17:00, 대전), (2012-04-14 17:00, 대전), (2012-04-14 17:00, 광주), (2012-04-14 17:00, 광주), (2012-04-14 17:00, 서울), (2012-04-14 17:00, 서울), (2012-04-14 17:00, 서울), (2012-04-14 17:00, 서울), (2012-04-15 14:00, 대전), (2012-04-15 14:00, 대전), (2012-04-15 14:00, 광주), (2012-04-15 14:00, 광주), (2012-04-15 14:00, 서울), (2012-04-15 14:00, 서울), (2012-04-15 14:00, 서울), (2012-04-15 14:00, 서울), (2012-04-17 19:00, 광주), (2012-04-17 19:00, 광주), (2012-04-17 19:00, 서울), (2012-04-17 19:00, 서울), (2012-04-17 19:00, 인천), (2012-04-17 19:00, 인천), (2012-04-17 19:00, 대구), (2012-04-17 19:00, 대구), (2012-04-18 19:00, 광주), (2012-04-18 19:00, 광주), (2012-04-18 19:00, 서울), (2012-04-18 19:00, 서울), (2012-04-18 19:00, 인천), (2012-04-18 19:00, 인천), (2012-04-18 19:00, 대구), (2012-04-18 19:00, 대구), (2012-04-19 19:00, 광주), (2012-04-19 19:00, 광주), (2012-04-19 19:00, 서울), (2012-04-19 19:00, 서울), (2012-04-19 19:00, 인천), (2012-04-19 19:00, 인천), (2012-04-19 19:00, 대구), (2012-04-19 19:00, 대구), (2012-04-20 19:00, 부산), (2012-04-20 19:00, 부산), (2012-04-20 19:00, 서울), (2012-04-20 19:00, 서울), (2012-04-20 19:00, 인천), (2012-04-20 19:00, 인천), (2012-04-20 19:00, 대구), (2012-04-20 19:00, 대구), (2012-04-22 14:00, 대구), (2012-04-22 14:00, 대구), (2012-04-24 19:00, 대전), (2012-04-24 19:00, 대전), (2012-04-24 19:00, 부산), (2012-04-24 19:00, 부산), (2012-04-24 19:00, 서울), (2012-04-24 19:00, 서울), (2012-04-24 19:00, 서울), (2012-04-24 19:00, 서울), (2012-04-26 19:00, 대전), (2012-04-26 19:00, 대전), ...]

[88592 rows x 0 columns]

In [5]:
data = pd.merge(baseball, weather, left_on= ['day_text','location'], right_on= ['day_text','location'], how='inner')

In [6]:
data

day times_text team_text  H_text  HR_text  \
day_text         location                                                
2012-04-07 14:00 대전         4.7토      13:55    한화 이글스      11        0   
                 대전         4.7토      13:55   롯데 자이언츠      10        1   
                 광주         4.7토      13:55  KIA 타이거즈       6        0   
                 광주         4.7토      13:55   SK 와이번스      10        0   
                 서울         4.7토      14:00    LG 트윈스       9        1   
...                          ...        ...       ...     ...      ...   
2012-09-30 17:00 부산        9.30일      17:00  KIA 타이거즈       2        1   
                 서울        9.30일      17:00    두산 베어스      14        0   
                 서울        9.30일      17:00    LG 트윈스       9        0   
                 대구        9.30일      17:00   삼성 라이온즈      13        1   
                 대구        9.30일      17:00   키움 히어로즈       7        0   

                           S_text  ST_text  DU_text  ER_text result_text   지점  \
day_text         location                                                       
2012-04-07 14:00 대전             2        6        2        2           패  133   
                 대전             2        6        1        1           승  133   
                 광주             2        8        0        3           패  156   
                 광주             1        2        1        0           승  156   
                 서울             1        3        2        0           승  108   
...                           ...      ...      ...      ...         ...  ...   
2012-09-30 17:00 부산             1        8        0        0           승  159   
                 서울             0        9        2        1           승  108   
                 서울             2        6        0        1           패  108   
                 대구             1        4        0        0           승  143   
                 대구             3        8        1        0           패  143   

                           기온(°C)  강수량(mm)  풍속(m/s)  풍향(16방위)  습도(%)  \
day_text         location                                              
2012-04-07 14:00 대전          12.8      NaN      2.4     340.0   17.0   
                 대전          12.8      NaN      2.4     340.0   17.0   
                 광주          14.6      NaN      2.3     270.0   24.0   
                 광주          14.6      NaN      2.3     270.0   24.0   
                 서울          11.1      NaN      4.1     250.0   19.0   
...                           ...      ...      ...       ...    ...   
2012-09-30 17:00 부산          21.1      NaN      2.4     140.0   57.0   
                 서울          19.8      NaN      3.3     290.0   49.0   
                 서울          19.8      NaN      3.3     290.0   49.0   
                 대구          21.9      NaN      1.7     340.0   39.0   
                 대구          21.9      NaN      1.7     340.0   39.0   

                           현지기압(hPa)  지면온도(°C)  
day_text         location                       
2012-04-07 14:00 대전           1013.5      23.2  
                 대전           1013.5      23.2  
                 광주           1013.9      29.5  
                 광주           1013.9      29.5  
                 서울           1010.8      27.6  
...                              ...       ...  
2012-09-30 17:00 부산           1000.0      24.4  
                 서울            999.3      20.2  
                 서울            999.3      20.2  
                 대구           1001.7      29.0  
                 대구           1001.7      29.0  

[1026 rows x 18 columns]

In [7]:
data.shape

(1026, 18)

In [8]:
data.to_csv('data/FinalData/2012_data.csv', index = True , encoding='utf-8-sig')